PostgreSQL provides you with many actions:
* Add a column
* Drop a column
* Change the data type of a column
* Rename a column
* Set a default value for the column
* Add a constraint to a column.
* Rename a table

# ADD COLUMN statement

```postgresql
ALTER TABLE table_name
ADD COLUMN column_name1 data_type constraint,
ADD COLUMN column_name2 data_type constraint,
...
ADD COLUMN column_namen data_type constraint;
```

**Adding a new column to a table**
```postgresql
ALTER TABLE customers
ADD COLUMN phone VARCHAR(25);
```

**Adding multiple columns to a table**
```postgresql
ALTER TABLE customers
ADD COLUMN fax VARCHAR (25),
ADD COLUMN email VARCHAR (400);
```

**Adding a column with a NOT NULL constraint to a table that already has data**
```postgresql
ALTER TABLE customers
ADD COLUMN contact_name VARCHAR(255) NOT NULL;
```

PostgreSQL issued an error: `ERROR:  column "contact_name" of relation "customers" contains null values`
* This is because the **contact_name** column has the `NOT NULL` constraint. 
* When PostgreSQL added the column, this new column received `NULL`, which violates the `NOT NULL` constraint.

To address this issue, you can follow these steps:
* First, add the **contact_name** column without the `NOT NULL` constraint.
* Second, update the values in the **contact_name** column.
* Third, modify the **contact_name** column to add the `NOT NULL` constraint.

```postgresql
-- First, add the contact_name column without the NOT NULL constraint:

ALTER TABLE customers
ADD COLUMN contact_name VARCHAR(255);

-- Second, update the values in the contact_name column.

UPDATE customers
SET contact_name = 'John Doe'
WHERE id = 1;
UPDATE customers
SET contact_name = 'Mary Doe'
WHERE id = 2;
UPDATE customers
SET contact_name = 'Lily Bush'
WHERE id = 3;

-- Third, modify the contact_name column to add the NOT NULL constraint:

ALTER TABLE customers
ALTER COLUMN contact_name
SET NOT NULL; 
```

# DROP COLUMN clause

```postgresql
ALTER TABLE table_name
DROP COLUMN column_name1,
DROP COLUMN column_name2,
...;
```

> It’s worth noting that while PostgreSQL allows a table that has no column, it may be not allowed according to the standard SQL.

When you remove a column from a table, PostgreSQL will automatically remove all of the **indexes** and **constraints** that involved the dropped column.

If the column that you want to remove is used in other database objects such as **views**, **triggers**, and **stored procedures**, you cannot drop the column because other objects depend on it.

In this case, you can use the `CASCADE` option in the `DROP COLUMN` clause to drop the column and all of its dependent objects:
```postgresql
ALTER TABLE table_name
DROP COLUMN column_name CASCADE; 
```

If you remove a column that does not exist, PostgreSQL will issue an error. 

To remove a column if it exists only, you can use the `IF EXISTS` option as follows:
```postgresql
ALTER TABLE table_name
DROP COLUMN IF EXISTS column_name; 
```

**Drop multiple columns example**
```postgresql
ALTER TABLE books
  DROP COLUMN isbn,
  DROP COLUMN description;
```

**Drop a column that is referenced by a constraint**
```postgresql
ALTER TABLE books
DROP COLUMN publisher_id;
```

PostgreSQL issued the following error:
```
ERROR:  cannot drop table books column publisher_id because other objects depend on it
DETAIL:  view book_info depends on table books column publisher_id
HINT:  Use DROP ... CASCADE to drop the dependent objects too.
```

The output states that the **book_info** view is using the column **publisher_id** of the **books** table. 

You need to use the `CASCADE` option to remove both the **publisher_id** column and **book_info** view as shown in the following statement:
```postgresql
ALTER TABLE books
DROP COLUMN publisher_id CASCADE; 
```

The statement issued the following notice indicating that the view **book_info** was also removed:
```postgresql
NOTICE:  drop cascades to view book_info
ALTER TABLE 
```

# Change Column Data Type

```postgresql
ALTER TABLE table_name
ALTER COLUMN column_name1 [SET DATA] TYPE new_data_type,
ALTER COLUMN column_name2 [SET DATA] TYPE new_data_type,
...; 
```

> The `SET DATA TYPE` and `TYPE` are equivalent.

PostgreSQL allows you to convert the values of a column to the new ones while changing its data type by adding a `USING` clause as follows:
```postgresql
ALTER TABLE table_name
ALTER COLUMN column_name TYPE new_data_type USING expression; 
```
The `USING` clause specifies an expression that allows you to convert the old values to the new ones.
* If you omit the `USING` clause, PostgreSQL will cast the values to the new ones implicitly. 
* If the cast fails, PostgreSQL will issue an error and recommend you provide the `USING` clause with an expression for the data conversion.

The expression after the `USING` keyword can be as simple as `column_name::new_data_type`, such as `price::numeric` or as complex as a custom function.

**Changing multiple columns example**
```postgresql
ALTER TABLE assets
    ALTER COLUMN location TYPE VARCHAR(255),
    ALTER COLUMN description TYPE VARCHAR(255);
```

**Changing a column from VARCHAR to INT example**
```postgresql
ALTER TABLE assets
ALTER COLUMN asset_no TYPE INT;
```

PostgreSQL issued an error and a helpful hint:
```
ERROR:  column "asset_no" cannot be cast automatically to type integer
HINT:  You might need to specify "USING asset_no::integer".
```

To both change the type of a column and cast data from `VARCHAR` to `INT`, you can use the `USING` clause:
```postgresql
ALTER TABLE assets
ALTER COLUMN asset_no TYPE INT
USING asset_no::integer;
```


# RENAME COLUMN clause

To rename multiple columns, you execute the `ALTER TABLE RENAME COLUMN` statement multiple times, one column at a time:
```postgresql
ALTER TABLE table_name
RENAME column_name1 TO new_column_name1;

ALTER TABLE table_name
RENAME column_name2 TO new_column_name2;
```
* For some reason, if you try to rename a column that does not exist, PostgreSQL will issue an error.
* Unfortunately, PostgreSQL does not support the `IF EXISTS` option for the `RENAME` clause.
* If you rename a column referenced by other database objects, such as views, foreign key constraints, triggers, and stored procedures, PostgreSQL will automatically change the column name in the dependent objects.

**Renaming one column example**
```postgresql
ALTER TABLE customers
RENAME COLUMN email TO contact_email; 
```

**Renaming a column that has dependent objects example**
```postgresql
ALTER TABLE customer_groups
RENAME COLUMN name TO group_name;
```

When you rename a column, PostgreSQL will automatically update its dependent objects, such as **foreign key** constraints, **views**, and **indexes**.

**Using multiple RENAME COLUMN statements to rename multiple columns example**

These statements rename two columns, **name** and **phone**, of the **customers** table to **customer_name** and **contact_phone**, respectively:
```postgresql
ALTER TABLE customers
RENAME COLUMN name TO customer_name;

ALTER TABLE customers
RENAME COLUMN phone TO contact_phone;
```

# RENAME TABLE statement

```postgresql
ALTER TABLE table_name
RENAME TO new_table_name;
```

If you rename a table that does not exist, PostgreSQL will issue an error.

To avoid the error, you can use the `IF EXISTS` option:
```postgresql
ALTER TABLE IF EXISTS table_name
RENAME TO new_table_name; 
```
* To rename multiple tables, you have to execute multiple `ALTER TABLE ... RENAME TO` statements.
* It’s not possible to rename multiple tables using a single `ALTER TABLE` statement.

**Renaming a table that has dependent objects**
```postgresql
ALTER TABLE customer_groups
RENAME TO groups;
```
When you rename a table, PostgreSQL will automatically update its dependent objects, such as **foreign key** constraints, **views**, and **indexes**.